In [13]:
import tensorflow as tf
from tqdm import tqdm

MODEL_SAVE_DIR = "savedModel"
FLOAT_LITE_MODEL_DIR = "floatLiteModel.tfl"
QUANT_LITE_MODEL_DIR = "quantLiteModel.tfl"
HEXE_LITE_MODEL_DIR = "quantLiteModel.cc"




In [14]:
mnist = tf.keras.datasets.mnist
(_, _), (x_test, y_test) = mnist.load_data()
x_test = x_test/255
x_test = x_test[..., tf.newaxis].astype("float32")

# validate the  data

In [15]:
import os


def get_dir_size(dir):
    size = 0
    for f in os.scandir(dir):
        if f.is_file():
            size += f.stat().st_size
        elif f.is_dir():
            size += get_dir_size(f.path)
    return size

# Calculate size
size_tf = get_dir_size(MODEL_SAVE_DIR)
size_no_quant_tflite = os.path.getsize(FLOAT_LITE_MODEL_DIR)
size_tflite = os.path.getsize(QUANT_LITE_MODEL_DIR)

print("original size:{}\nno quant size:{}\nquant size:{}".format(size_tf,size_no_quant_tflite,size_tflite))

original size:78816
no quant size:19880
quant size:7104


In [16]:
import numpy as np
def predict_tflite(tflite_model, img):
    img_array = tf.expand_dims(img, 0)

    # Initialize the TFLite interpreter
    interpreter = tf.lite.Interpreter(model_path=tflite_model)
    interpreter.allocate_tensors()

    input_details = interpreter.get_input_details()[0]
    output_details = interpreter.get_output_details()[0]

    # If required, quantize the input layer (from float to integer)
    input_scale, input_zero_point = input_details["quantization"]
    if (input_scale, input_zero_point) != (0.0, 0):
        img_array = np.multiply(img_array, 1.0 / input_scale) + input_zero_point
        img_array = img_array.astype(input_details["dtype"])

    # Invoke the interpreter
    interpreter.set_tensor(input_details["index"], img_array)
    interpreter.invoke()
    pred = interpreter.get_tensor(output_details["index"])[0]

    # If required, dequantized the output layer (from integer to float)
    output_scale, output_zero_point = output_details["quantization"]
    if (output_scale, output_zero_point) != (0.0, 0):
        pred = pred.astype(np.float32)
        pred = np.multiply((pred - output_zero_point), output_scale)

    predicted_label_index = np.argmax(pred)
    predicted_score = pred[predicted_label_index]
    return (predicted_label_index, predicted_score)

In [17]:
def run_tflite_test(model_file):
    correct_count = 0
    wrong_count = 0
    discarded_count = 0
    for i,j in enumerate(x_test):
        index, score = predict_tflite(model_file, x_test[i])
        if score < 0.75:
            discarded_count += 1
            continue
        if index == y_test[i]:
            correct_count += 1
        else:
            wrong_count += 1
            print("[%s] expected, [%s] found with score [%f]" % (y_test[i], index, score))

    correct_percentage = (correct_count / (correct_count + wrong_count)) * 100


    print("%.1f%% correct (N=%d, %d unknown)" % (correct_percentage, (correct_count + wrong_count), discarded_count))

In [18]:
run_tflite_test(FLOAT_LITE_MODEL_DIR)

[4] expected, [0] found with score [4.540332]
[4] expected, [8] found with score [1.730161]
[6] expected, [0] found with score [8.353472]
[2] expected, [3] found with score [3.020800]
[9] expected, [7] found with score [1.061835]
[2] expected, [7] found with score [4.422511]
[5] expected, [3] found with score [3.023057]
[3] expected, [7] found with score [5.523836]
[5] expected, [3] found with score [6.315693]
[4] expected, [5] found with score [1.818221]
[6] expected, [0] found with score [4.633207]
[8] expected, [2] found with score [2.554385]
[7] expected, [9] found with score [5.246146]
[2] expected, [8] found with score [4.278044]
[1] expected, [8] found with score [3.462673]
[2] expected, [1] found with score [3.379570]
[5] expected, [3] found with score [5.391839]
[7] expected, [3] found with score [4.568510]
[0] expected, [5] found with score [1.062623]
[5] expected, [8] found with score [5.966059]
[7] expected, [5] found with score [2.212738]
[2] expected, [8] found with score

In [19]:
run_tflite_test(QUANT_LITE_MODEL_DIR)

[4] expected, [0] found with score [4.234404]
[4] expected, [8] found with score [1.693762]
[6] expected, [0] found with score [8.468808]
[2] expected, [3] found with score [3.175803]
[9] expected, [7] found with score [1.270321]
[2] expected, [7] found with score [4.657845]
[5] expected, [3] found with score [3.387523]
[3] expected, [7] found with score [5.716446]
[5] expected, [3] found with score [6.351606]
[6] expected, [0] found with score [4.657845]
[8] expected, [2] found with score [2.117202]
[2] expected, [8] found with score [2.540642]
[8] expected, [3] found with score [1.905482]
[7] expected, [9] found with score [5.716446]
[2] expected, [8] found with score [4.234404]
[1] expected, [8] found with score [3.387523]
[2] expected, [1] found with score [3.599244]
[5] expected, [3] found with score [5.716446]
[7] expected, [3] found with score [4.869565]
[0] expected, [5] found with score [0.846881]
[5] expected, [8] found with score [5.716446]
[7] expected, [5] found with score

# transform the data into json

In [7]:
savedData = {}
import numpy as np
interpreter = tf.lite.Interpreter(model_path=QUANT_LITE_MODEL_DIR)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
input_scale, input_zero_point = input_details["quantization"]
output_scale, output_zero_point = output_details["quantization"]
for i,j in enumerate(x_test):
    img_array = np.multiply(tf.reshape(j,[784]), 1.0 / input_scale) + input_zero_point
    img_array = img_array.astype(input_details["dtype"])
    savedData[i] = {"data":img_array.tolist(),"label":int(y_test[i])}



for i in savedData:
    interpreter.set_tensor(input_details["index"], savedData[i]["data"])
    interpreter.invoke()
    pred = interpreter.get_tensor(output_details["index"])[0]
    if (output_scale, output_zero_point) != (0.0, 0):
        pred = pred.astype(np.float32)
        pred = np.multiply((pred - output_zero_point), output_scale)

    predicted_label_index = np.argmax(pred)
    predicted_score = pred[predicted_label_index]
    print(savedData[i]["label"], predicted_label_index)
print(savedData[0])


In [ ]:
partData = {i:{'data':savedData[i]['data'],'label':savedData[i]['label']} for i in range(100)}

In [10]:
import json
with open("test.json","w") as file:
    json.dump(partData,file)


In [8]:
type(savedData[1]['label'])

int

In [11]:
output_scale, output_zero_point

(0.21172021329402924, 24)

In [33]:
partData

NameError: name 'partDa' is not defined

In [28]:
savedData = {}
import numpy as np
interpreter = tf.lite.Interpreter(model_path=QUANT_LITE_MODEL_DIR)
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]
input_scale, input_zero_point = input_details["quantization"]
output_scale, output_zero_point = output_details["quantization"]
for i,j in enumerate(x_test):
    img_array = np.multiply(tf.expand_dims(j,0), 1.0 / input_scale) + input_zero_point
    img_array = img_array.astype(input_details["dtype"])
    savedData[i] = {"data":img_array.tolist(),"label":int(y_test[i])}

for i in savedData:
    interpreter.set_tensor(input_details["index"], np.array(savedData[i]["data"]).astype("int8"))
    interpreter.invoke()
    pred = interpreter.get_tensor(output_details["index"])[0]
    # if (output_scale, output_zero_point) != (0.0, 0):
    #     pred = pred.astype(np.float32)
    #     pred = np.multiply((pred - output_zero_point), output_scale)
    print(pred)
    break
    #
    # predicted_label_index = np.argmax(pred)
    # predicted_score = pred[predicted_label_index]
    # print(savedData[i]["label"], predicted_label_index)


[  5 -11  23  21 -40 -13 -94  83   5   4]


In [29]:
output_details['dtype']

numpy.int8